In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3.0,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10.0,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10.0,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10.0,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10.0,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
143,DECEMBER,23.0,Top Gun: Maverick,Paramount Pictures / Skydance Media / Don Simp...,"Joseph Kosinski (director); Ehren Kruger, Eric...",[142]
144,DECEMBER,23.0,The Croods 2,Universal Pictures / DreamWorks Animation,"Joel Crawford (director); Kevin Hageman, Dan H...",[143]
145,DECEMBER,25.0,The Last Duel,20th Century Studios / Scott Free Productions ...,"Ridley Scott (director); Ben Affleck, Matt Dam...",[144]
146,DECEMBER,25.0,News of the World,Universal Pictures / Columbia Pictures,Paul Greengrass (director/screenplay); Luke Da...,[145]


In [10]:
df_2020 = df[['Title','Cast and crew']]

In [11]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...
...,...,...
143,Top Gun: Maverick,"Joseph Kosinski (director); Ehren Kruger, Eric..."
144,The Croods 2,"Joel Crawford (director); Kevin Hageman, Dan H..."
145,The Last Duel,"Ridley Scott (director); Ben Affleck, Matt Dam..."
146,News of the World,Paul Greengrass (director/screenplay); Luke Da...


In [12]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '5492165c61b1a21c06eb3a3b578a6339'

In [13]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        genre_list = str(genres)
        return genre_list
#         return genre_str.join(genre_list)
    else:
        np.NaN

In [14]:
df_2020['Genre'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-14-03673a9a4aeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['Genre'] = df_2020['Title'].map(lambda x: get_genre(str(x)))


In [15]:
df_2020

,Title,Cast and crew,Genre
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,"['Horror', 'Mystery']"
1,Underwater,"William Eubank (director); Brian Duffield, Ada...","['Horror', 'Science Fiction', 'Action', 'Thril..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",['Comedy']
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,"['Drama', 'Thriller', 'Crime']"
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,"['Horror', 'Thriller', 'Mystery']"
...,...,...,...
143,Top Gun: Maverick,"Joseph Kosinski (director); Ehren Kruger, Eric...","['Action', 'Drama']"
144,The Croods 2,"Joel Crawford (director); Kevin Hageman, Dan H...","['Animation', 'Adventure', 'Family']"
145,The Last Duel,"Ridley Scott (director); Ben Affleck, Matt Dam...",['Drama']
146,News of the World,Paul Greengrass (director/screenplay); Luke Da...,"['Drama', 'Western']"


In [16]:
def list_to_string(x):
    if x:
        return x.replace(',', '').replace('[','').replace(']','').replace("'",'')
    else:
        return np.NaN

In [17]:
df_2020['genres'] = df_2020['Genre'].map(lambda x: list_to_string(x))

<ipython-input-17-0a2f4ad09b6e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Genre'].map(lambda x: list_to_string(x))


In [18]:
df_2020

,Title,Cast and crew,Genre,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,"['Horror', 'Mystery']",Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...","['Horror', 'Science Fiction', 'Action', 'Thril...",Horror Science Fiction Action Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",['Comedy'],Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,"['Drama', 'Thriller', 'Crime']",Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,"['Horror', 'Thriller', 'Mystery']",Horror Thriller Mystery
...,...,...,...,...
143,Top Gun: Maverick,"Joseph Kosinski (director); Ehren Kruger, Eric...","['Action', 'Drama']",Action Drama
144,The Croods 2,"Joel Crawford (director); Kevin Hageman, Dan H...","['Animation', 'Adventure', 'Family']",Animation Adventure Family
145,The Last Duel,"Ridley Scott (director); Ben Affleck, Matt Dam...",['Drama'],Drama
146,News of the World,Paul Greengrass (director/screenplay); Luke Da...,"['Drama', 'Western']",Drama Western


In [19]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [20]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-20-bc52fe9b0c35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [21]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [22]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [23]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [24]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [25]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [26]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [27]:
df_2020

,Title,Cast and crew,Genre,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,"['Horror', 'Mystery']",Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...","['Horror', 'Science Fiction', 'Action', 'Thril...",Horror Science Fiction Action Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",['Comedy'],Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,"['Drama', 'Thriller', 'Crime']",Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,"['Horror', 'Thriller', 'Mystery']",Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer
...,...,...,...,...,...,...,...,...
143,Top Gun: Maverick,"Joseph Kosinski (director); Ehren Kruger, Eric...","['Action', 'Drama']",Action Drama,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly
144,The Croods 2,"Joel Crawford (director); Kevin Hageman, Dan H...","['Animation', 'Adventure', 'Family']",Animation Adventure Family,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds
145,The Last Duel,"Ridley Scott (director); Ben Affleck, Matt Dam...",['Drama'],Drama,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer
146,News of the World,Paul Greengrass (director/screenplay); Luke Da...,"['Drama', 'Western']",Drama Western,Paul Greengrass,Tom Hanks,Helena Zengel,Neil Sandilands


In [28]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [29]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [30]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Thriller,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,Inherit the Viper
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,The Sonata
...,...,...,...,...,...,...
143,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,Top Gun: Maverick
144,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,The Croods 2
145,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,The Last Duel
146,Paul Greengrass,Tom Hanks,Helena Zengel,Neil Sandilands,Drama Western,News of the World


In [32]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [33]:
new_df20['actor_2_name'] = new_df20['actor_2_name'].replace(np.nan, 'unknown')
new_df20['actor_3_name'] = new_df20['actor_3_name'].replace(np.nan, 'unknown')
new_df20['director_name'] = new_df20['director_name'].replace(np.nan, 'unknown')
new_df20['movie_title'] = new_df20['movie_title'].replace(np.nan, 'unknown')
new_df20['actor_1_name'] = new_df20['actor_1_name'].replace(np.nan, 'unknown')
new_df20['comb'] = new_df20['comb'].replace(np.nan, 'unknown')
new_df20['genres'] = new_df20['genres'].replace(np.nan, 'unknown')

In [34]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [35]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Drama Thriller Crime,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
4,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer,Horror Thriller Mystery,the sonata,Freya Tingley Simon Abkarian Rutger Hauer Andr...
...,...,...,...,...,...,...,...
143,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Jose...
144,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
145,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scot...
146,Paul Greengrass,Tom Hanks,Helena Zengel,Neil Sandilands,Drama Western,news of the world,Tom Hanks Helena Zengel Neil Sandilands Paul G...


In [40]:
new_df20.to_csv('transformed_wiki_2020.csv',index=False)